# THICC: Tennessee's Hottest Industries by County Calculator

This notebook utilizes the Agency for Healthcare Research and Quality (AHRQ)'s Social Determinants of Health (SDOH) Database to train a Random Forest regression model to predict the best Tennessee counties for employment given a user's job industry, education level, and other socioeconomic variables. The user will see the top 5 best Tennessee counties and the overall employment landscape in Tennessee for a given industry.

## Data Preprocessing & Cleaning

This project utilizes AHRQ SDOH data and the U.S. Census Bureau's census tract data. Due to the large file sizes, all spreadsheets, CSVs, and Shapefiles can be found on Harvard Dataverse (https://doi.org/10.7910/DVN/GIOAZY).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import collections

pd.set_option('display.max_columns', None) 

### Load in AHRQ SDOH data

The original AHRQ SDOH data comes in Excel spreadsheet format. The spreadsheet contains two sheets: LAYOUT and DATA. The LAYOUT sheet contains the column variable names and written descriptions. The DATA sheet contains the data and statistics for those column variable names. 

In [ ]:
path = 'data/sdoh_2020_tract_1_0.xlsx'
dfs = pd.read_excel(
        path,
        sheet_name = None,
        dtype = str,
        na_filter = False,
        nrows = 1000,
)

for name, df in dfs.items():
    print(name)
    with pd.option_context('display.max_columns', None):
        display(df.head())

layout_df = dfs['Layout']

Let's look at the LAYOUT sheet and identify the data types of the columns.

In [ ]:
def scope():
    nonnumerics = []
    for name, row in layout_df.iterrows():
        if row['type'] != 'num':
            nonnumerics.append(name)

    for name in nonnumerics:
        print(f'        {name!r}: str,')

scope()

Now we know to identify some of the columns as strings instead of floats. Reading the entire DATA sheet as an Excel spreadsheet takes on average 3 minutes. To speed up the process, let's use the CSV version of the DATA sheet instead.

In [ ]:
path = 'data/sdoh_2020_tract_1_0_data.csv'
data_df = pd.read_csv(
    path,
    dtype = collections.defaultdict(lambda: float) | {
        'TRACTFIPS': str,
        'COUNTYFIPS': str,
        'STATEFIPS': str,
        'STATE': str,
        'COUNTY': str,
        'REGION': str,
        'CEN_AIAN_NH_IND': str,
    },
    na_filter = True,
    na_values=['', ' '],
)
display(data_df.head())

Let's print out some information on the data in our DataFrame.

In [ ]:
#Display info on the data columns
data_df.info()

#Generate statistics on the data columns
df.describe()

### Merge AHRQ SDOH DataFrame with TN Census Tract shapefile
The U.S. Census Bureau has a Census Tract shapefile for Tennessee in 2020, so we can give the Tennessee counties a geometry to plot to.
Let's merge our AHRQ DataFrame with the Census Tract GeoPandas GeoDataFrame (GDF) by the matching TRACTFIPS and GEOID respectively.

In [ ]:
shapefile = gpd.read_file('data/tl_2020_47_tract/tl_2020_47_tract.shp')
display(shapefile.head())

In [ ]:
#After performing the merge
merged = data_df.merge(shapefile[['GEOID', 'geometry']], left_on='TRACTFIPS', right_on='GEOID', how='left')

#Reorder the columns to move 'GEOID' and 'geometry' to the front
columns = ['GEOID', 'geometry'] + [col for col in merged.columns if col not in ['GEOID', 'geometry']]
merged = merged[columns]

gdf = gpd.GeoDataFrame(merged, geometry='geometry')
tennessee_gdf = gdf[gdf["GEOID"].astype(str).str.startswith('47')]
display(tennessee_gdf.head())

Now that the GDF has all the information, let's filter it down to just the columns we need. In particular, we keep columns with county/geographic information, median age, average household income, percentage of population education, and percentage of population with specific industry jobs.

In [ ]:
filtered_tennessee_gdf = tennessee_gdf[
    [
        'GEOID',
        'geometry',
        'STATE',
        'COUNTY',
        'COUNTYFIPS',
        'CEN_POPDENSITY_TRACT',
        'ACS_TOT_CIVILIAN_LABOR',
        'ACS_TOT_CIVIL_EMPLOY_POP',

        'ACS_MEDIAN_AGE',


        'ACS_PCT_LT_HS',
        'ACS_PCT_HS_GRADUATE',
        'ACS_PCT_COLLEGE_ASSOCIATE_DGR',   
        'ACS_PCT_BACHELOR_DGR',
        'ACS_PCT_GRADUATE_DGR',
        'ACS_PCT_POSTHS_ED',
        
        'ACS_MEDIAN_HH_INC',
        'ACS_MEDIAN_INC_F',
        'ACS_MEDIAN_INC_M',
        
        'ACS_PCT_EMPLOYED',
        'ACS_PCT_NOT_LABOR',
        'ACS_PCT_UNEMPLOY',

        'ACS_PCT_ADMIN',
        'ACS_PCT_ARMED_FORCES',
        'ACS_PCT_ART',
        'ACS_PCT_CONSTRUCT',
        'ACS_PCT_EDUC',
        'ACS_PCT_FINANCE',
        'ACS_PCT_GOVT',
        'ACS_PCT_INFORM',
        'ACS_PCT_MANUFACT',
        'ACS_PCT_NATURE',
        'ACS_PCT_OTHER',
        'ACS_PCT_PROFESS',
        'ACS_PCT_RETAIL',
        'ACS_PCT_TRANSPORT',
        'ACS_PCT_WHOLESALE',
    ]
]

display(filtered_tennessee_gdf.head())

Now that we filtered our GDF, let's look at what kind of data we're handling.

In [ ]:
filtered_tennessee_gdf.info()

filtered_tennessee_gdf.describe()

### Clean the data
Let's remove empty values and preprocess the data. First, let's check how many empty values there are.

In [ ]:
old_shape = filtered_tennessee_gdf.shape
print("GDF shape: ", old_shape)

highest_empty_count = 0
for i in filtered_tennessee_gdf:
    if filtered_tennessee_gdf[i].isna().sum() != 0:
        print(f"{i}: {filtered_tennessee_gdf[i].isna().sum()}")
    if filtered_tennessee_gdf[i].isna().sum() > highest_empty_count:
        highest_empty_count = filtered_tennessee_gdf[i].isna().sum()

print(highest_empty_count)

Out of 1700 rows of data, dropping ~19 rows will not be a lot of lost data. Let's drop the empty values.

In [ ]:
clean_filtered_tennessee_gdf = filtered_tennessee_gdf.dropna()
deleted_rows = old_shape[0] - clean_filtered_tennessee_gdf.shape[0]
print(f"Old shape: {old_shape}")
print(f"New shape: {clean_filtered_tennessee_gdf.shape}")
print(f"Deleted rows: {deleted_rows}")

## Create Random Forest model
Now that our data is ready to be used, let's create a predictor model. A random forest is an ensemble method that creates multiple decision trees by bootstrapping data samples and using random subsets of features for each tree. The final prediction is made by averaging the outputs of all trees, reducing overfitting and improving accuracy compared to a single decision tree.

i should explain here why random forest is a good choice.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

In [ ]:
df = clean_filtered_tennessee_gdf.copy()

# Drop geometry column for ML training
df_ml = df.drop(columns=['geometry'])

# Create dictionary mappings for career fields to relevant features
career_field_features = {
    'armed forces' : 'ACS_PCT_ARMED_FORCES',
    'arts (entertainment, recreation, accommodation, and food services)' : 'ACS_PCT_ART',
    'construction' : 'ACS_PCT_CONSTRUCT',
    'educational services (healthcare, social assistance)' : 'ACS_PCT_EDUC',
    'finance (insurance, real estate, rental/leasing)' : 'ACS_PCT_FINANCE',
    'government' : 'ACS_PCT_GOVT',
    'informaton services' : 'ACS_PCT_INFORM',
    'manufacturing' : 'ACS_PCT_MANUFACT',
    'nature (agriculature, forestry, fishing, hunting, mining)' : 'ACS_PCT_NATURE',
    'other': 'ACS_PCT_OTHER',
    'professional (scientific, management, administrative, and waste management)' : 'ACS_PCT_PROFESS',
    'public administration' : 'ACS_PCT_ADMIN',
    'retail' : 'ACS_PCT_RETAIL',
    'transportation (warehousing, utilities)' : 'ACS_PCT_TRANSPORT',
    'wholesale' : 'ACS_PCT_WHOLESALE',
}

# Create education level features
education_levels = {
    'less_than_high_school': 'ACS_PCT_LT_HS',
    'high_school': 'ACS_PCT_HS_GRADUATE',
    'some_college/associates': 'ACS_PCT_COLLEGE_ASSOCIATE_DGR',
    'bachelors': 'ACS_PCT_BACHELOR_DGR',
    'masters/doctorate': 'ACS_PCT_GRADUATE_DGR',   
    'postsecondary': 'ACS_PCT_POSTHS_ED'
}


feature_names = [
    'ACS_MEDIAN_HH_INC', 
    'ACS_MEDIAN_INC_F',
    'ACS_MEDIAN_INC_M',

    'ACS_TOT_CIVILIAN_LABOR',
    'ACS_TOT_CIVIL_EMPLOY_POP',

    'ACS_PCT_LT_HS',
    'ACS_PCT_HS_GRADUATE',
    'ACS_PCT_COLLEGE_ASSOCIATE_DGR',
    'ACS_PCT_BACHELOR_DGR',
    'ACS_PCT_GRADUATE_DGR',
    'ACS_PCT_POSTHS_ED',

    'ACS_PCT_ARMED_FORCES',
    'ACS_PCT_ART',
    'ACS_PCT_CONSTRUCT',
    'ACS_PCT_EDUC',
    'ACS_PCT_FINANCE',
    'ACS_PCT_GOVT',
    'ACS_PCT_INFORM',
    'ACS_PCT_MANUFACT',
    'ACS_PCT_NATURE',
    'ACS_PCT_OTHER',
    'ACS_PCT_PROFESS',
    'ACS_PCT_ADMIN',
    'ACS_PCT_RETAIL',
    'ACS_PCT_TRANSPORT',
    'ACS_PCT_WHOLESALE'
]

### Train the RF model
Let's create a function to train a random forest model to predict industry suitability scores for counties. Creating a score that factors in household income, population density, and other variables will make it easier for the regression model to predict and return something.

In [ ]:
def train_career_hotspot_rf_model():
    feature_vectors = []
    labels = []
    df = clean_filtered_tennessee_gdf.copy()

    scaler = MinMaxScaler() 
    df.loc[:, feature_names] = scaler.fit_transform(df[feature_names])


    # Loop through each county
    for _, county in df.iterrows():
        features = [county.get(f, 0) for f in feature_names]
        feature_vectors.append(features)
        
        # Create a composite target that represents overall career suitability
        target_score = 0        
        for f in feature_names:
            target_score += county.get(f, 0)
        
        #for career, column in career_field_features.items():
        #    target_score += county.get(column, 0) 
        
        
        labels.append(target_score)


    #Split the data into training 70% and testing 30%
    X = np.array(feature_vectors)
    y = np.array(labels)
    x_train_reg, x_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.33, random_state=42)

    
    #Train model with cross-validation to avoid overfitting
    model = RandomForestRegressor(
        n_estimators=100, 
        max_depth=10,  # Prevent overfitting
        min_samples_split=5,
        random_state=42
    )
    #Check model performance with cross-validation
    cv_scores = cross_val_score(model, x_train_reg, y_train_reg, cv=5, scoring='r2')
    print(f"Cross-validation R² scores: {cv_scores}")
    print(f"Mean R² score: {cv_scores.mean():.3f}")
    
    #Do the real training/testing split
    model.fit(x_train_reg, y_train_reg)
    y_pred_reg = model.predict(x_test_reg)
    mae = mean_absolute_error(y_test_reg, y_pred_reg)
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    mape = mean_absolute_percentage_error(y_test_reg, y_pred_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)

    print("Mean Average Error (MAE) score: ", mae)
    print("Mean Squared Error (MSE) score: ", mse)
    print("Mean Absolute Percentage Error (MAPE) score : ", mape)
    print("R² score: ", r2)


    

    return model, feature_names

### Predict with RF model
Now that we trained our RF model, let's predict industry suitability scores for all Tennessee counties given user-inputted industry name and education level.

In [ ]:
def predict_with_model_for_industry(model, feature_names, selected_career, selected_education_level):
    # Prepare feature matrix
    features_matrix = clean_filtered_tennessee_gdf[feature_names].fillna(0).values

    # Base predictions from the model (this gives the general career suitability score)
    base_preds = model.predict(features_matrix)

    # Create prediction DataFrame
    prediction_df = clean_filtered_tennessee_gdf[['GEOID', 'COUNTY', 'COUNTYFIPS', selected_career, selected_education_level]].copy()
    prediction_df['predicted_score'] = base_preds


    '''
    This approach ensures that counties with particularly strong metrics for your selected career and education level get ranked higher than they might in the general prediction.
    For example, if you're interested in healthcare careers for people with bachelor's degrees:

    Some counties might score well overall but not have strong healthcare employment
    Others might have excellent healthcare employment but lower scores in other areas
    Your boosting would help identify counties that are both generally good and specifically strong for healthcare + bachelor's degree holders

    The 70/30 weighting (70% base prediction, 30% career/education factors) maintains balance so you don't completely disregard general livability while still emphasizing your specific interests.
        
    '''

    # Focus on the selected career/industry for boosting
    if selected_career in career_field_features:
        career_column = career_field_features[selected_career]
        if career_column in clean_filtered_tennessee_gdf.columns:
            #Create ratio to see if percentage is above or below average
            career_boost = clean_filtered_tennessee_gdf[career_column] / clean_filtered_tennessee_gdf[career_column].mean()
            #The formula gives 70% weight to the original score and 30% to the career-specific boost
            prediction_df['predicted_score'] *= (0.7 + 0.3 * career_boost.fillna(0))

    # If education level is selected, you could also adjust the score for education-specific features, if desired
    if selected_education_level in education_levels:
        education_column = education_levels[selected_education_level]
        if education_column in clean_filtered_tennessee_gdf.columns:
            #Create ratio to see if percentage is above or below average
            education_boost = clean_filtered_tennessee_gdf[education_column] / clean_filtered_tennessee_gdf[education_column].mean()
            #The formula gives 70% weight to the original score and 30% to the career-specific boost
            prediction_df['predicted_score'] *= (0.7 + 0.3 * education_boost.fillna(0))


    # Normalize to 0-100 range
    min_score = prediction_df['predicted_score'].min()
    max_score = prediction_df['predicted_score'].max()
    prediction_df['predicted_score'] = 100 * (prediction_df['predicted_score'] - min_score) / (max_score - min_score)

    # Return the sorted dataframe based on the predicted score, highest first
    return prediction_df.sort_values(by='predicted_score', ascending=False)


## Implementation
With the model created, let's add interactivity to it and plot the results.

In [ ]:
print("\nTraining Random Forest model...")
model, feature_names = train_career_hotspot_rf_model()

In [ ]:
importances = model.feature_importances_
sorted_indices = np.argsort(importances)[::-1]

print("\nFeature importances:")
for idx in sorted_indices:
    print(f"{idx}: {feature_names[idx] if idx < len(feature_names) else 'Career Feature'} - {importances[idx]:.4f}")

In [ ]:
plt.figure(figsize=(12, 8))
plt.barh([feature_names[idx] if idx < len(feature_names) else f"Career Feature {idx}" for idx in sorted_indices],
         importances[sorted_indices])
plt.xlabel("Feature Importance")
plt.title("Feature Importances from Random Forest")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
print("\nAvailable career fields:")
for i, career in enumerate(career_field_features.keys(), 1):
    print(f"{i}. {career.title()}")
career_idx = int(input("\nEnter the number for your desired career field: ")) - 1
career_field = list(career_field_features.keys())[career_idx]
career_value = career_field_features[career_field]


# Display available education levels
print("\nAvailable education levels:")
for i, edu_level in enumerate(education_levels.keys(), 1):
    print(f"{i}. {edu_level.replace('_', ' ').title()}")
edu_idx = int(input("\nEnter the number for your education level: ")) - 1
education_level = list(education_levels.keys())[edu_idx]    
education_value = education_levels[education_level]


predictions = predict_with_model_for_industry(model, feature_names, career_value, education_value)

predictions = predictions.sort_values('predicted_score', ascending=False)
print(f"\nTop 10 recommended counties for {career_field.title()} career (ML predictions):")
top_counties = predictions.head(10)
display(top_counties)



tn_plot_gdf = pd.merge(
    clean_filtered_tennessee_gdf,
    predictions[['GEOID', 'predicted_score']],
    on=['GEOID'],
    how='left'  
)

In [ ]:
import plotly.express as px
import numpy as np

# Create the figure with hover data
fig = px.choropleth_map(
    tn_plot_gdf,
    geojson=tn_plot_gdf.geometry.__geo_interface__,
    locations=tn_plot_gdf.index,
    color='predicted_score',
    color_continuous_scale='Blues',
    hover_data=['GEOID', 'COUNTY', 
                'ACS_TOT_CIVIL_EMPLOY_POP', 'ACS_MEDIAN_HH_INC', 'ACS_PCT_POSTHS_ED',
                'predicted_score'],
    opacity=0.8,
    center={"lat": 35.86, "lon": -86.36},
    zoom=7,
    map_style="carto-positron",
    labels={'predicted_score': 'Predicted Score'}
)

# Update layout
fig.update_layout(
    title='Predicted Scores by Census Tract',
    margin={"r": 0, "t": 30, "l": 0, "b": 0},
    height=700,
    coloraxis_colorbar=dict(
        title="Score",
    )
)

# Update hover template
fig.update_traces(
    hovertemplate='<b>GEOID: %{customdata[0]}</b><br>' +
                  'County: %{customdata[1]}<br>' +
                  'Total Employed Population: %{customdata[2]}<br>' +
                  'Household Income: $%{customdata[3]}<br>' +
                  'Postsecondary Education: %{customdata[4]}%<br>' +
                  'Predicted Score: %{customdata[5]:.2f}<br>'
)

fig.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
tn_plot_gdf.plot(
    column='predicted_score',
    cmap='viridis', 
    legend=True,
    legend_kwds={'label': "Suitability Score", 'orientation': "horizontal"},
    ax=ax
)
plt.title(f'Career Suitability Scores for {career_field.title()} with {education_level.replace("_", " ").title()}', fontsize=16)
plt.axis('off')
plt.tight_layout()
#plt.savefig('career_suitability_map.png', dpi=300)
plt.show()